In [1]:
import dspy
from dspy.evaluate import Evaluate
from dspy.datasets.hotpotqa import HotPotQA
from dspy.teleprompt import BootstrapFewShotWithRandomSearch, BootstrapFinetune


c:\Users\m.landoulsi\Desktop\pprojects\fun\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ports = [7140, 7141, 7142, 7143, 7144, 7145]
llamaChat = llama3 = lm = dspy.OllamaLocal(model='llama3')
colbertv2 = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

dspy.settings.configure(rm=colbertv2, lm=llamaChat)

In [3]:
dataset = HotPotQA(train_seed=1,train_size=200,eval_seed=2023,dev_size=1000,test_size=0)
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]
testset = [x.with_inputs('question') for x in dataset.test]

c:\Users\m.landoulsi\Desktop\pprojects\fun\venv\Lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [4]:
len(trainset), len(devset), len(testset)

(200, 1000, 0)

In [5]:
trainset[0]

Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'})

In [6]:
from dsp.utils.utils import deduplicate

class BasicMultiHop(dspy.Module):
    def __init__(self,passages_per_hop=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        self.generate_query = [ dspy.ChainOfThought("context, question -> search_query") for _ in range(2)]
        self.generate_answer = dspy.ChainOfThought("context, question -> answer")

    def forward(self,question):
        context = []
        for hop in range(2):
            search_query = self.generate_query[hop](context=context,question=question).search_query
            passages = self.retrieve(search_query).passages
            context = deduplicate(context + passages)

        return self.generate_answer(context=context,question=question).copy(context=context)


In [9]:
RECOMPILE_INTO_LLAMA_FROM_SCRATCH = True
NUM_THREADS = 4

metric_EM = dspy.evaluate.answer_exact_match

In [11]:
if RECOMPILE_INTO_LLAMA_FROM_SCRATCH:
    teleprompter = BootstrapFewShotWithRandomSearch(metric=metric_EM,max_bootstrapped_demos=2,num_threads=NUM_THREADS)
    basicMultiHop_cmp = teleprompter.compile(BasicMultiHop(),trainset=trainset[:50],valset=trainset[50:200])
    ensemble = [prog for *_ , prog in basicMultiHop_cmp.candidate_programs[:4]]

    for idx, prog in enumerate(ensemble):
        prog.save(f'examples/checkpoints/multihop_llama38b_{idx}.json')


  0%|          | 0/150 [00:00<?, ?it/s]

In [1]:
if not RECOMPILE_INTO_LLAMA_FROM_SCRATCH:
    ensemble = []

    for idx in range(4):
        prog = BasicMultiHop()
        prog.load(f'checkpoints/multihop_llama38b_{idx}.json')
        ensemble.append(prog)
        

NameError: name 'RECOMPILE_INTO_LLAMA_FROM_SCRATCH' is not defined

In [ ]:
llama_program = ensemble[0]

evaluate_hotpot = Evaluate(devset=devset[:1000],metric=metric_EM,num_threads=NUM_THREADS,display_progress=True,display_table=0)
evaluate_hotpot(llama_program)

In [ ]:
llama_program(question="How many storeys are in the castle that David Gregory inherited?")

llamaChat.inspect_history(n=3)

In [ ]:
unlabeled_train = HotPotQA(train_seed=1, train_size=3000, eval_seed=2023, dev_size=0, test_size=0).train
unlabeled_train = [dspy.Example(question=x.question).with_inputs('question') for x in unlabeled_train]
len(unlabeled_train)
